<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.1: Your First Chisel Module
**Prev: [Introduction to Scala](1_intro_to_scala.ipynb)**<br>
**Next: [Combinational Logic](2.2_comb_logic.ipynb)**

## Motivation
现在你已经熟悉了Scala，让我们开始设计一些硬件吧！Chisel代表着在Scala嵌入语言中构建硬件。这意味着它是Scala中的DSL，允许你在同一代码中充分利用Scala和Chisel编程。重要的是要理解哪些代码是“Scala”和哪些代码是“Chisel”，但我们稍后会更详细地讨论这一点。暂时将Chisel和Module 2中的代码视为编写Verilog的更好方法。此模块向你展示了一个完整的Chisel `Module`和测试器。现在只是大致了解一下。你将在后面看到更多的例子。

## Setup
以下单元格下载了Chisel所需的依赖项。你将在所有未来的笔记本中看到它。**现在运行此单元格**。

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

正如在上一个模块中提到的，需要这些语句来导入Chisel。**现在运行此单元格**，然后再运行任何未来的代码块。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

---
# Your First Module
本节将介绍你的第一个硬件模块、一个测试案例以及如何运行它。它将包含许多你可能不理解的内容，但没关系。我们希望你能掌握大致的情况，以便你可以不断返回到这个完整且可工作的示例中，巩固你学到的知识。

<span style="color:blue">**示例：一个模块**</span><br>
与Verilog一样，我们可以在Chisel中声明模块定义。以下示例是一个Chisel `Module`，`Passthrough`，它有一个4位输入`in`和一个4位输出`out`。该模块将`in`和`out`组合连接，因此`in`驱动`out`。

In [ ]:
// Chisel Code: 声明一个新的模块定义
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

这里有很多内容！以下是如何从描述的硬件角度思考每一行的说明。

```scala
class Passthrough extends Module {
```
我们声明一个名为`Passthrough`的新模块。`Module`是Chisel的内置类，所有硬件模块都必须扩展它。

```scala 
val io = IO(...)
```
我们在一个特殊的`io` `val`中声明所有输入和输出端口。它必须被称为`io`，并且是一个`IO`对象或实例，其形式必须是`IO(_instantiated_bundle_)`。

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
我们声明一个新的硬件结构类型（Bundle），其中包含一些命名的信号`in`和`out`，它们的方向分别为Input和Output。

```scala
UInt(4.W)
```
我们声明信号的硬件类型。在这种情况下，它是一个宽度为4的无符号整数。

```scala
io.out := io.in
```
我们连接我们的输入端口到输出端口，以使`io.in` *驱动* `io.out`。请注意，`:=`运算符是一个***Chisel***运算符，表示右侧信号驱动左侧信号。它是一个有方向的运算符。

硬件构造语言（HCLs）的妙处在于我们可以将底层编程语言用作脚本语言。例如，在声明了Chisel模块之后，我们可以使用Scala调用Chisel编译器将Chisel `Passthrough`翻译成Verilog `Passthrough`。这个过程称为***elaboration***。

In [ ]:
// Scala Code: 通过将其翻译成Verilog来详细说明我们的Chisel设计
// 不要担心理解这段代码；这是非常复杂的Scala
println(getVerilog(new Passthrough))

<span style="color:blue">**示例：一个模块生成器**</span><br>
如果我们将在Scala中学到的内容应用于这个例子，我们可以看到Chisel模块是作为Scala类实现的。就像任何其他Scala类一样，我们可以使Chisel模块接受一些构造参数。在这种情况下，我们创建一个新的类`PassthroughGenerator`，它将接受一个整数`width`，该整数指定其输入和输出端口的宽度：

In [ ]:
// Chisel Code，但传递参数以设置端口的宽度
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// 现在让我们生成不同宽度的模块
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

请注意，生成的Verilog使用不同的位宽来匹配`width`参数分配的值。让我们深入了解这是如何工作的。由于Chisel模块是普通的Scala类，我们可以使用Scala类构造函数的强大功能来参数化我们设计的详细过程。

您可能会注意到，这种参数化是由*Scala*启用的，而不是*Chisel*；Chisel没有用于参数化的额外API，但设计师可以简单地利用Scala功能来参数化他/她的设计。

由于`PassthroughGenerator`不再描述单个模块，而是描述由`width`参数参数化的一组模块，我们将此`Passthrough`称为***生成器***。

---
# 测试您的硬件

没有测试器的硬件模块或生成器都不应该是完整的。Chisel具有内置的测试功能，您将在本培训中探索这些功能。以下示例是一个Chisel测试硬件的测试框架，该测试框架向`Passthrough`的输入端口`in`传递值，并检查输出端口`out`上是否看到相同的值。

<span style="color:blue">**示例：一个测试器**</span><br>
这里发生了一些高级的Scala。但是，除了`poke`和`expect`命令之外，你不需要理解任何其他内容。你可以将代码的其余部分简单地看作编写这些简单测试的样板代码。

In [ ]:
// Scala Code: `test`运行单元测试。 
// test接受一个用户模块，并有一个代码块，该代码块将pokes和expects应用于
// 测试电路（c）
test(new Passthrough()) { c =>
    c.io.in.poke(0.U)     // 将我们的输入设置为值0
    c.io.out.expect(0.U)  // 断言输出正确为0
    c.io.in.poke(1.U)     // 将我们的输入设置为值1
    c.io.out.expect(1.U)  // 断言输出正确为1
    c.io.in.poke(2.U)     // 将我们的输入设置为值2
    c.io.out.expect(2.U)  // 断言输出正确为2
}
println("SUCCESS!!") // Scala Code: 如果我们到达这里，我们的测试通过了！


发生了什么？测试接受一个`Passthrough`模块，为模块的输入分配值，并检查其输出。要设置输入，我们调用`poke`。要检查输出，我们调用`expect`。如果我们不想将输出与期望值进行比较（没有断言），我们可以使用`peek`来查看输出。

如果所有`expect`语句都为真，则我们的样板代码将返回通过。

>请注意，`poke` 和 `expect` 使用 Chisel 硬件文字表示法。这两个操作都期望正确类型的文字字面值。
If `poke`ing a `UInt()` you must supply a `UInt` literal (example: `c.io.in.poke(10.U)`, likewise if the input is a `Bool()` the `poke` would expect either `true.B` or `false.B`.



<span style="color:red">**练习：编写自己的测试器**</span><br>
编写并执行两个测试，一个测试 `PassthroughGenerator` 的宽度为 10，另一个测试 `PassthroughGenerator` 的宽度为 20。至少检查两个值：零和指定宽度支持的最大值。请注意，在Scala中，三个问号具有特殊含义。在这些训练营练习中，你可能经常看到它。使用 `???` 运行代码将产生 `NotImplementedError`。请用你自己的代码替换 `???`。

In [ ]:
// 使用宽度 10 进行测试

test(???) { c =>
    ???
}

// 使用宽度 20 进行测试

test(???) { c =>
    ???
}

println("SUCCESS!!") // Scala 代码：如果我们到达这里，我们的测试通过了！

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>解决方案</strong>（单击切换显示）</label>
<article>
<pre style="background-color:#f7f7f7">
test(new PassthroughGenerator(10)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1023.U)
    c.io.out.expect(1023.U)
}

test(new PassthroughGenerator(20)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.io.in.poke(1048575.U)
    c.io.out.expect(1048575.U)
}

</pre></article></div></section></div>

---
# 查看生成的 Verilog/FIRRTL

如果你对生成的硬件有困惑，并且能够阅读结构化的 Verilog 和/或 FIRRTL（类似于 Verilog 的合成-only 子集的 Chisel IR），那么你可以尝试查看生成的 Verilog，以查看 Chisel 执行的结果。

这是一个生成 Verilog（你已经看过的）和 FIRRTL 的示例。

In [ ]:
// 用于调试查看 Verilog
println(getVerilog(new Passthrough))

In [ ]:
// 用于调试查看 FIRRTL
println(getFirrtl(new Passthrough))

---
# 完成了！

[返回顶部](#top)

## <span style="color:red"> 附录：关于 "printf" 调试的注释</span>
[使用打印语句进行调试](https://stackoverflow.com/a/189570) 不总是调试的最佳方法，但通常是查看出现问题时发生了什么的简单第一步。
因为 Chisel 生成器是生成硬件的程序，所以在打印生成器和电路状态时有一些额外的微妙之处。
在执行打印语句时记住什么时候执行以及正在打印什么是很重要的。
在这里，你可能需要使用不同的打印方式：
* Chisel 生成器在电路生成过程中打印
* 电路在电路模拟期间打印
* Tester 在测试期间打印

`println` 是Scala的内置函数，用于在控制台打印。它**不能**用于在电路模拟期间打印，因为生成的电路是 FIRRTL 或 Verilog，而不是 Scala。

以下代码块显示了不同的打印样式。

In [ ]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("Print during simulation: Input is %d\n", io.in)
    // chisel printf has its own string interpolator too
    printf(p"Print during simulation: IO is $io\n")

    println(s"Print during generation: Input is ${io.in}")
}

test(new PrintingModule ) { c =>
    c.io.in.poke(3.U)
    c.clock.step(5) // circuit will print
    
    println(s"Print during testing: Input is ${c.io.in.peek()}")
}